In [144]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [145]:
#import Plotly 
import plotly.express as px

In [146]:
#print head of cleaned dataset
DatabaseDF = pd.read_csv("../CleanedDataset.csv")
DatabaseDF.head()

,Unnamed: 0,Operator,Well Name,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs",...,Fluid Type from DI,SPACING_CAPPED,12 month Cum Prod,Completion Type,Sleeves,P&P,Frac Ports,Repeater Ports,CT,Year Drilled
0,0,Operator 1,Well 1,MBH,5114.0,30.0,141,99,10,0.0,...,NaN,660.0,33.55,NaN,NaN,NaN,NaN,NaN,NaN,2006.0
1,1,Operator 2,Well 2,MBH,5181.0,NaN,142,99,16,418000.0,...,NaN,660.0,27.62,NaN,NaN,NaN,NaN,NaN,NaN,2005.0
2,2,Operator 2,Well 3,MBH,4780.0,15.0,142,99,24,568407.0,...,NaN,660.0,11.19,OH,NaN,NaN,NaN,NaN,NaN,2006.0
3,3,Operator 1,Well 4,MBH,10654.0,1.0,142,99,1,845000.0,...,NaN,660.0,27.85,NaN,NaN,NaN,NaN,NaN,NaN,2007.0
4,4,Operator 2,Well 5,MBH,5209.0,1.0,142,99,35,297366.0,...,NaN,660.0,50.88,OH,NaN,NaN,NaN,NaN,NaN,2006.0


In [147]:
#Get unique completion types 
DatabaseDF['Completion Type'].unique()
completion_types = ['OH', 'Sleeves', 'P & P', 'Sleeves and P & P', 'Screen',
       'Cemented Liner', 'Perforated Liner', 'Frac Ports',
       'Frac Ports and P & P', 'Frac Ports and Repeater Ports', 'CT',
       'P & P and CT', 'Repeater Ports and P & P']

In [148]:
#Create new columns for each completion type
def add_type(feature_name, unique_type):
    new_column = []
    for item in DatabaseDF[feature_name]:
        if item == unique_type:
            new_column.append(item)
        else:
            new_column.append('NaN')
    return new_column

In [149]:
#Add to dataframe
for i in completion_types: 
    DatabaseDF[i] = add_type('Completion Type', i)

In [150]:
#Get unique completion types 
DatabaseDF['Formation'].unique()
formation_types = ['MBH', 'TFH', 'MBH/TFH', 'TF2', 'TF4', 'TF3', 'UTFH', 'TF1',
       'MTFH', 'TFSH', 'TF2.5']

In [151]:
#Add to dataframe
for i in formation_types: 
    DatabaseDF[i] = add_type('Formation', i)

In [152]:
#Get unique formation types 
DatabaseDF['Fluid Type from DI'].unique()
fluid_types = ['LINEAR GEL (HC)', 'CROSSLINKED (HC)', 'SLICKWATER (HC)',
       'SW-XL HYBRID', 'CROSSLINKED', 'UNKNOWN', 'SW-GEL HYBRID',
       'LINEAR GEL']

In [153]:
#Add to dataframe
for i in fluid_types: 
    DatabaseDF[i] = add_type('Fluid Type from DI', i)

In [154]:
def convert_var(feature_list):
    #Change categorical variables to numerical values
    for i in feature_list:
        labels = DatabaseDF[i].unique().tolist()
        mapping = dict(zip(labels,range(len(labels))) )
        DatabaseDF.replace({i: mapping},inplace=True)   

In [156]:
convert_var(completion_types)

In [157]:
convert_var(formation_types)

In [158]:
convert_var(fluid_types)

In [160]:
def numerical_conversion(feature_name):
    #Change categorical variables to numerical values
    labels = DatabaseDF[feature_name].unique().tolist()
    mapping = dict( zip(labels,range(len(labels))) )
    DatabaseDF.replace({feature_name: mapping},inplace=True)   

In [161]:
features_toconvert = ['Completion Type', 'Formation', 'Operator', 'Fluid Type from DI', 'Date Fracd']
for i in features_toconvert:
    numerical_conversion(i)

# Spider (Radar) Chart: Definition

A radar chart is used primarily in fields such as Business to compare various quantitative variables in a dataset. The chart has multiple radii, where each radius represents one variable. The magnitude of the variable is represented by the length of the radius. The longer the radius, the more of an effect the variable has. In this notebook specifically, a longer radius represents a variable more correlated to the cumulative production.

The Radar Chart is a useful tool to visualize how the features in our dataset affect each other and the oil production, and will help us with feature selection because we can narrow down the variables and focus on the ones that seem the most impactful. 

This notebook will use different correlation coefficients, such as Pearson, Spearman's and Kendall, to analyze variables with the radar charts.

# Spearman's Rank Correlation

### Spearman's Rank was the correlation that was chosen because it best represents the relationships presented within this dataset.

In [176]:
#Spearman's Rank correlation coefficient for each feature
spearman_correlation = DatabaseDF.corr(method = 'spearman')
spearman_correlation.head()

,Unnamed: 0,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",...,TFSH,TF2.5,LINEAR GEL (HC),CROSSLINKED (HC),SLICKWATER (HC),SW-XL HYBRID,CROSSLINKED,UNKNOWN,SW-GEL HYBRID,LINEAR GEL
Unnamed: 0,1.000000,0.000309,-0.047491,0.077660,0.232800,0.381223,0.006677,-0.049775,0.243052,0.198040,...,0.001260,0.022539,0.057226,0.019071,0.061320,0.004895,-0.020744,-0.030026,0.038083,0.000620
Operator,0.000309,1.000000,-0.071951,-0.063621,0.072341,-0.085731,-0.077516,0.025940,0.076770,0.036760,...,0.017927,0.007002,0.006320,0.002745,0.028635,0.023183,0.001318,-0.021736,-0.013262,0.014481
Formation,-0.047491,-0.071951,1.000000,0.115587,0.102739,0.015774,0.047869,0.011154,0.032733,0.084415,...,0.040043,0.028328,0.017349,0.089642,0.038847,0.024101,-0.006071,-0.019788,-0.037903,0.012641
Lateral Length,0.077660,-0.063621,0.115587,1.000000,0.435665,0.018391,0.181624,-0.030477,0.326196,0.307917,...,0.026483,-0.009038,0.078654,0.061182,0.109113,0.009412,-0.005954,-0.004929,-0.000686,0.011437
Stages,0.232800,0.072341,0.102739,0.435665,1.000000,0.052379,0.162301,0.029076,0.658696,0.630845,...,0.023003,-0.006736,0.159467,0.125547,0.227036,0.021138,-0.007845,-0.016621,0.073999,0.005156


In [163]:
#Printing the correlation to Cumulative Production specifically
spearman_correlation['12 month Cum Prod']

Unnamed: 0                       0.145759
Operator                         0.134709
Formation                       -0.090278
Lateral Length                   0.177257
Stages                           0.340166
Township                        -0.165085
Range                           -0.157996
Section                          0.010521
Total Prop, lbs                  0.440060
Fluid, bbls                      0.401960
Date Fracd                       0.245793
Best1 Mo BOPD                    0.891192
Best3 Mo BOPD                    0.937187
Best6 Mo BOPD                    0.968343
Best9 Mo BOPD                    0.977030
Best12 Mo BOPD                   0.977382
Latitude                        -0.166509
Longitude                        0.175363
Fluid Type from DI               0.162423
SPACING_CAPPED                  -0.246285
12 month Cum Prod                1.000000
Completion Type                  0.229161
Sleeves                         -0.007928
P&P                              0

## First, we start with testing all of the quantitative variables together in two different groups.

### Group 1

In [164]:
#Features that are being compared
features = ['Lateral Length', 'Stages', 'Fluid, bbls', 'Total Prop, lbs', 'Longitude']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [165]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

It seems that proppant and fluid seem to have the higher correlation values, at around 0.4-0.45. This means that these variables have a higher, stronger association with cumulative production and are more important to the contribution towards production.

### Group 2

In [166]:
#Features that are being compared
features = ['Latitude','CT','Section',
           'Year Drilled', 'SPACING_CAPPED']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [167]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

Year Drilled has the highest correlation coefficient in this set, at 0.3. This is not as high as the previous set of variables but it is still worth looking into how the Year Drilled affects production and how related these two factors are.

## Next, all of the categorical variables are compared together.

In [168]:
#Features that are being compared
features = ['Completion Type', 'Formation', 'Fluid Type from DI', 'Date Fracd', 'Operator']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [169]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

Completion Type and Date Fracd are most correlated to cumulative preduction, but the coefficent is only around 0.2 to 0.25, which is not as high as some of the other features in the dataset.

## Then, we analyze the data more by comparing the unique types within each categorical variable.

### Completion Types

In [170]:
#Features that are being compared
features = completion_types
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [171]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

It seems like P&P seems to have the highest correlation to the cumulative production out of all the completion types. This means that using a P&P completion type might result in higher oil production.

### Formation Types

In [172]:
#Features that are being compared
features = formation_types
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [173]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

TF1 and TFSH are the two types of formations that have the highest correlation to cumulative production. This means that using either of these Formation types might result in more oil production.

### Fluid Types

In [174]:
#Features that are being compared
features = fluid_types
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [175]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

The Slickwater Fluid seems to be the fluid type most correlated with cumulative production when comparing it to the other fluid types. Even though the correlation value is lower than some of the other features, using this type of fluid could result in higher oil production.